## Software Engineering

In [106]:
# 계산기를 모듈화하여 구현
import time


class InputInterfaceModule:
    def check_inputs(self, x, y, operator, verbose):
        assert x.isnumeric(), "x is must be integer or float. not string or any"
        assert y.isnumeric(), "y is must be integer or float. not string or any"
        assert operator in ["+","-","*","/","//","%"], "operator is must be one of +,-,*,/,//,%"
        if verbose:
            print("expression confirmed")

class ResultInterfaceModule:
    def print_result(self, x, y, operator, result):
        print_result = f"Result: {x} {operator} {y} = {result}"
        print("="*len(print_result))
        print(print_result)
        print("="*len(print_result))

class CalculateModule(ResultInterfaceModule):
    def calculate(self, x, y, operator, verbose):
        if operator == "+":
            result = x + y
        elif operator == "-":
            result = x - y
        elif operator == "*":
            result = x * y
        elif operator == "/":
            result = x / y
        elif operator == "//":
            result = x // y
        elif operator == "%":
            result = x % y
        
        if verbose:
            self.print_result(x, y, operator, result)
        
        return result

class InputModule(InputInterfaceModule):
    def split_expression(self, expression, verbose):
        x, operator, y = expression.split()
        self.check_inputs(x, y, operator, verbose)  # check expression
        return int(x), int(y), operator

class Calculator(InputModule, CalculateModule):
    def __init__(self, verbose=1):
        self.verbose = verbose
        
        if self.verbose:
            print("Calculator is ready.")
        
    def execute(self, expression:str):
        if self.verbose:
            print("executing...")
        
        x, y, operator = self.split_expression(expression, self.verbose)
        return self.calculate(x, y, operator, self.verbose)

calculator = Calculator(verbose=1)
result = calculator.execute("1 * 1")

Calculator is ready.
executing...
expression confirmed
Result: 1 * 1 = 1


#### with 문

- @contextmanager, yield
- class의 \_\_enter\_\_, \_\_exit\_\_

In [11]:
import time
import contextlib

# 데코레이터 사용
@contextlib.contextmanager
def timing():
    print("yield 윗 부분 실행 됐지롱")
    t0 = time.time()  # with 문 시작할 때 실행
    yield
    print("yield 아랫 부분 실행 됐지롱")
    t1 = time.time()  # with 문 끝날 때 실행
    print(t1 - t0)

# 클래스 사용
class timer:
    def __init__(self):
        pass
    
    def __enter__(self):  # with 문 시작할 때 실행
        print("enter 시작 됐지롱")
        self.start = time.time()
    
    def __exit__(self, a, b, c):  # with 문 끝날 때 실행
        print("exit 시작 됐지롱")
        print(time.time() - self.start)

with timing():
    print("Hello")
    time.sleep(1)

time.sleep(0.5)
print("="*10)
time.sleep(0.5)

with timer():
    print("World")
    time.sleep(1)


yield 윗 부분 실행 됐지롱
Hello
yield 아랫 부분 실행 됐지롱
1.0012731552124023
enter 시작 됐지롱
World
exit 시작 됐지롱
1.0024480819702148


#### yield

- 파이썬은 사용자 정의 함수 def 안에 yield가 있으면 generator 라고 취급한다.
- def + return = function
- def + yield = generator

In [11]:
def range_r(number):
    for num in range(number):
        print("r")
        return num

def range_s(number):
    for num in range(number):
        print("s")
        yield num

range_r(10)
range_s(10)

r


<generator object range_s at 0x7fd943068270>

- generator 는 반복문을 이용해서 사용한다.

In [15]:
for item in range_s(10):
    print(f"item = {item}")

s
item = 0
s
item = 1
s
item = 2
s
item = 3
s
item = 4
s
item = 5
s
item = 6
s
item = 7
s
item = 8
s
item = 9


- 여기서 중요한 것은 반복문이 실행 되면서 매 번 item을 가져올 때, yield가 실행된 시점을 기억한다는 점이다.
    - ex) 첫번째 item을 가져오고 난 후 -> range_s(): "첫번째 반복, i=0이었고, yield 0을 했음"을 기억하고 다음 반복땐 yield가 끝난 지점부터 다시 시작한다.

- yield 를 여러번 선언하여 반복문을 통해 순차적으로 선언된 값을 출력할 수도 있다.

In [10]:
def yield_1():
    yield 1
    yield 12
    yield 123

for item in yield_1():
    print(item)

1
12
123


- 더이상 실행할 yield가 없거나 중간에 return 을 만난다면? -> generator 로써의 역할을 마침

In [12]:
# 이미 실행된 generator를 재사용하는 경우
def yield_1():
    yield 1
    yield 12
    yield 123

function = yield_1()

for item in function:
    print(item)

# function = yield_1()  # 한번 더 선언 해줘야 실행 가능

for item in function:  # 실행 안됨
    print(item)

1
12
123


In [14]:
# 중간에 return 이 선언된 경우
def yield_2():
    yield 1
    return 12
    yield 123  # pylance extension이 있다면 비활성화 표시가 보임

for item in yield_2():
    print(item)

1


- generator에서 item을 하나씩 접근하고 싶다면? -> next() 함수 사용

In [26]:
def yield_3():
    yield 1
    yield 12
    yield 123

function = yield_3()

print(f"item = {next(function)}")
print(f"item = {next(function)}")
print(f"item = {next(function)}")

item = 1
item = 12
item = 123


- @contextmanager 와 yield를 사용하여 실사용 예제를 수행해보자.

In [9]:
# 예시) 밑에 === 로 구분해놓은 함수 부분을 utils.py 라는 파일에 넣어 놓고
#      from utils import write_open 이런식으로 불러와 사용할 수 있다.
# ========================================
import os
from contextlib import contextmanager

@contextmanager
def write_open(path):
    dir_name = os.path.dirname(path)
    if not os.path.isdir(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    writer = open(path, "w")
    print(1)  # 첫 번째 실행
    
    # -----------
    yield writer  # 기준
    # -----------
    
    writer.close()
    print(3)  # 세 번째 실행
# ========================================

with write_open("./test/test.txt") as f:
    f.write("test")
    print(2)  # 두 번째 실행
print(4)  # 네 번째 실행

2
4


#### lru_cache()

- 정의: "그거 이미 해봄ㅋ", "안돼"

In [32]:
import time
from functools import lru_cache

@lru_cache()
def think(idea):
    print(f"Hmm... {idea}")
    time.sleep(3)
    return "안돼"

- 이제 think 함수에 "pineapple pizza"와 "mint chocolate"을 전달 해보자.

In [33]:
print(think("pineapple pizza"))
print(think("mint chocolate"))

Hmm... pineapple pizza
안돼
Hmm... mint chocolate
안돼


- 위 두 번의 출력은 lru_cache()가 선언 되었든 아니든 상관 없이 기능상 문제가 없다.
- 그런데 여기서 think 함수에 "pineapple pizza"를 한번 더 전달한다면? -> 묻지도 따지지도 않고 return 값인 "안돼"를 반환한다. "mint chocolate"도 마찬가지이다.

In [36]:
print(think("pineapple pizza"))
print(think("pineapple pizza"))

print(think("mint chocolate"))
print(think("mint chocolate"))

안돼
안돼
안돼
안돼


- 이것이 바로 lru_cache()의 기능이다. 이미 결과를 구했던 입력에 대해서 다음번에 호출할 때에는 이미 구했었던 결과를 바로 리턴해준다.
- 이를 있어 보이는 말로 메모이제이션(memoization)이라고 부른다.
- 그렇다면 이런 기능을 실제로는 어떤 때에 활용할 수 있을까? -> 대표적으로 코딩테스트에서 DP(Dynamic Programming)문제를 풀 때, 사용 되는 기법이다.
- 또다른 편법은 lru_cache()를 재귀 함수와 함께 사용하면, 메모이제이션 효과를 내어 전체 실행 시간을 비약적으로 단축 시킬 수 있다.

In [20]:
# lru_cache() 미사용 -> SSAP 느림
import time


def fibonacci(x):
    if x <= 1:
        return x
    return fibonacci(x-1) + fibonacci(x-2)

start = time.time()
answer = fibonacci(35)
end = time.time()

print(end - start)
print(answer)

17.020562887191772
9227465


In [27]:
# lru_cache() 사용 -> 이게 내가 알던 재귀함수가 맞나
import time
from functools import lru_cache


@lru_cache()
def fibonacci(x):
    if x <= 1:
        return x
    return fibonacci(x-1) + fibonacci(x-2)

start = time.time()
for i in range(71):
    print(fibonacci(i), end=" ")
end = time.time()

print()
print(end - start)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 190392490709135 
0.0005571842193603516


In [28]:
# 메모이제이션 직접 구현 버젼 -> 이게 내가 알던 재귀함수가 맞나..22
import time


def fibonacci(x, cache=dict()):
    if x in cache:
        return cache[x]
    if x <= 1:
        return x
    cache[x] = fibonacci(x-1) + fibonacci(x-2)
    return fibonacci(x-1) + fibonacci(x-2)

start = time.time()
for i in range(71):
    print(fibonacci(i), end=" ")
end = time.time()

print()
print(end - start)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 190392490709135 
0.0008327960968017578


In [29]:
# 리스트를 활용한 메모이제이션 기법 적용 -> 사실상 n값이 커질 수록 이 방법이 가장 효율적
import time


n = 70
start = time.time()
f = [0, 1] + [0]*(n-1)
for i in range(2, n+1):
    f[i] = f[i-1] + f[i-2]
for i in range(n+1):
    print(f[i], end=" ")
end = time.time()

print()
print(end - start)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 20365011074 32951280099 53316291173 86267571272 139583862445 225851433717 365435296162 591286729879 956722026041 1548008755920 2504730781961 4052739537881 6557470319842 10610209857723 17167680177565 27777890035288 44945570212853 72723460248141 117669030460994 190392490709135 
0.0005519390106201172


#### Multi processing

In [1]:
from square import square
import time


start = time.time()

for i in range(10000):
    print(f"\rs = {square(i)}", end="")

end = time.time()
print(f"\ntime = {end-start}")

s = 99980001
time = 13.04424786567688


In [2]:
from multiprocessing import Pool, cpu_count
from square import square
import time


n_cpu = cpu_count()
print("Number of CPU:", n_cpu)
pools = Pool(n_cpu-1)

start = time.time()

for i in pools.imap(square, range(10000)):
    print(f"\rs = {i}", end="")

end = time.time()
print(f"\ntime = {end-start}")

Number of CPU: 12


s = 99980001
time = 1.2655439376831055


#### 딕셔너리 활용법

In [1]:
# 라이브러리 미사용
text = "The policeman whose name is JOHN is going to the doughnut store "+\
    "JAMES aka doughnut master greeted JOHN and poke to JOHN"

words = text.split(" ")
name_count = {}
for word in words:
    if word.isupper():
        if word in name_count:
            name_count[word] += 1
        else:
            name_count[word] = 1
        
print(name_count)

{'JOHN': 3, 'JAMES': 1}


In [2]:
# defaultdict 라이브러리 사용
from collections import defaultdict
text = "The policeman whose name is JOHN is going to the doughnut store "+\
    "JAMES aka doughnut master greeted JOHN and poke to JOHN"

words = text.split(" ")
# key가 없어서 에러가 발생하는 상황이라면 name_count[key] = 0을 몰래 미리 수행한다.
name_count = defaultdict(int) # or lambda: n 으로 임의의 초기값을 설정할 수도 있다.
print(name_count)
for word in words:
    if word.isupper():
        name_count[word] += 1
        
print(dict(name_count))

defaultdict(<class 'int'>, {})
{'JOHN': 3, 'JAMES': 1}


In [28]:
# defaultdict 없이 직접 구현 -> 잔기술
text = "The policeman whose name is JOHN is going to the doughnut store "+\
    "JAMES aka doughnut master greeted JOHN and poke to JOHN"

words = text.split(" ")
name_count = {}
for word in words:
    if word.isupper():
        name_count.get(word) or name_count.update({word:0})  # and/or 연산자의 특성: 앞에서 이미 답이 정해졌다면, 뒤는 보지 않는다.
        name_count[word] += 1
        
print(name_count)

{'JOHN': 3, 'JAMES': 1}


In [ ]:
d = {1:1}
if d.get(1) or 9/0:
    print(True)

True


In [3]:
d = {}
if d.get(1) and 9/0:
    print(True)

In [62]:
d = {}
a = "apple"
a == "apple" and d.update({2:2})
# if a == "apple":
#   d.update({2:2})
d

{2: 2}

In [63]:
a = "apple"
b = ""
a == "apple" and (b := "pineapple")
# if a == "apple":
#   d.update({2:2})
b

'pineapple'

In [72]:
a = None
b = None
class Node:
    def __init__(self, value=0, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

class Solution:
    def isSubtree(self, root, subRoot):
        if root is None or subRoot is None:
            return root is None and subRoot is None
        
        return self.is_equal(root, subRoot) or self.isSubtree(root.left, subRoot) or self.isSubtree(root.right, subRoot)

    def is_equal(self, root, subRoot):
        if root is None or subRoot is None:
            return root is None and subRoot is None
        
        return root.value == subRoot.value and self.is_equal(root.left, subRoot.left) and self.is_equal(root.right, subRoot.right)

True

In [36]:
def calculate_intersection_over_union(bounding_box_a, bounding_box_b, epsilon=1e-6):
    '''
    PASCAL VOC format:
    bounding_box = [class, confidence, x_min, y_min, x_max, y_max]
    '''
    
    # intersection
    x1 = max(bounding_box_a[2], bounding_box_b[2])
    y1 = max(bounding_box_a[3], bounding_box_b[3])
    x2 = min(bounding_box_a[4], bounding_box_b[4])
    y2 = min(bounding_box_a[5], bounding_box_b[5])

    width = x2 - x1
    height = y2 - y1
    
    if width < 0 or height < 0:
        return 0.0
    
    overlapped_area = width * height
    
    # union
    bounding_box_a_width = bounding_box_a[4] - bounding_box_a[2]
    bounding_box_a_height = bounding_box_a[5] - bounding_box_a[3]
    area_bounding_box_a = bounding_box_a_width * bounding_box_a_height
    
    bounding_box_b_width = bounding_box_b[4] - bounding_box_b[2]
    bounding_box_b_height = bounding_box_b[5] - bounding_box_b[3]
    area_bounding_box_b = bounding_box_b_width * bounding_box_b_height
    
    combined_area = area_bounding_box_a + area_bounding_box_b - overlapped_area
    
    intersection_over_union = overlapped_area / (combined_area+epsilon)
    return round(intersection_over_union, 4)

prediction_bbox = [200, 200, 400, 400]
ground_truth_bbox = [250, 250, 450, 450]

iou = calculate_intersection_over_union(prediction_bbox, ground_truth_bbox)
print(iou)

0.3913


In [4]:
def descending_confidences(detections):
    '''
    detection = [TP(1) or FP(0), confidence]
    detections = [detection1, detection2, ...]
    '''
    result = []
    # insertion sort
    for detection in detections:
        result.append(detection)
        if len(result) > 1 and result[-2][1] < result[-1][1]:
            i = -1
            while i != -len(result) and result[i-1][1] < result[i][1]:
                result[i-1], result[i] = result[i], result[i-1]
                i -= 1
    return result

detections = [[1, 0.58], [1, 0.95], [0, 0.5], [0, 0.12], [1, 0.33]]
print(detections)
print(descending_confidences(detections))

[[1, 0.58], [1, 0.95], [0, 0.5], [0, 0.12], [1, 0.33]]
[[1, 0.95], [1, 0.58], [0, 0.5], [1, 0.33], [0, 0.12]]


In [2]:
def get_precision(true_positives, detections, epsilon=1e-6):
    return round(true_positives / (detections+epsilon), 4)

def get_recall(true_positives, ground_truths, epsilon=1e-6):
    return round(true_positives / (ground_truths+epsilon), 4)

def get_precision_recall_curve(detections, ground_truths):
    new_detections = []
    for 
    pass


[[1, 0.95], [1, 0.58], [0, 0.5], [1, 0.33], [0, 0.12]]

In [7]:
detections = [[1, 0.58], [1, 0.95], [0, 0.5], [0, 0.12], [1, 0.33]]
for i in range(-1, -11, -1):
    print(i)
    if i == -len(detections):
        break


-1
-2
-3
-4
-5


In [16]:
def forward(x, f, h, returns):
    xf = x + f
    fh = f + h
    xfh = xf + fh
    result = xfh // 2
    
    variables = locals()
    return [variables[r] for r in returns]

a, b = forward(1, 2, 3, ["xf", "result"])
print(a, b)

3 4


In [17]:
# r: 공비
# a_1: 초항
# a_n: n차항
r = 3
a_1 = 1
a_n = 27
answer = (r * a_n - a_1) // (r - 1)
print(answer)

40
